# Query Examples for SDK

Some of the most common query operators in the Sight Machine SDK.

In [1]:
from smsdk import client
from datetime import datetime, timedelta
import pandas as pd

In [2]:
cli = client.Client('tenant_name')
cli.login('apikey', 
          key_id = 'key_name', 
          secret_id = 'secret_name')

types = cli.get_machine_type_names()

machine_type = types[0]
machines = cli.get_machine_names(source_type=machine_type)

columns = cli.get_machine_schema(machines[0])['display'].to_list()

## A basic starting query.

This is the same query as from the quick start, used to set a baseline.  Then we'll use variations on this theme to demonstrate different query options and their effects.

Note that this baseline query already demonstrates:
- Basic filter rules formatted as key value pairs
- Filtering for greater than or less than values
    - It uses `__gte` for greater than or equal.  Use `__gt` for greater than.  Similarly `__lte` is less than or equal vs. `__lt` for less than.
- Sorting returned results
    - Note the `-` prefix before `Endtime` means to sort descending.  To sort ascending, do not place a prefix in front of the variable name.

In [3]:
query = {'Machine': machines[0],
         'End Time__gte' : datetime(2020, 9, 1), 
         'End Time__lte' : datetime(2020, 9, 2), 
         '_order_by': '-End Time'}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')
df.head()

_only not specified.  Selecting first 50 fields.
Size of returned data: (1441, 40)


,Start Time,End Time,Cycle Time (Net),Cycle Time (Gross),Shift,Machine,BM: Axial Load,BM: Dome Depth,BM: Thickwall Variation,BM: Trim Height Max,...,BM: Thinwall Avg,BM: Trim Height Avg,0_BM: CPM,BM: Trim Height Variation,0_BM: Primary Tear Off PPM,BM: Trimmer Jam - Data,BM: Trimmer Jam Time,BM: Trimmer Jam Events,BM: Cans Out,BM: Cans Out Total
id,,,,,,,,,,,,,,,,,,,,,
5f6eeb3e5d20e300018cb3dc,2020-09-01 23:59:00,2020-09-02 00:00:00,60000,60000,Day,BM 1,86.75,11.15,14.6,131.2865,...,89.450002,131.246999,256.000000,0.08,0.0,0.0,0,0,None,2920.0
5f6eeb3e5d20e300018cb3db,2020-09-01 23:58:00,2020-09-01 23:59:00,60000,60000,Day,BM 1,86.75,11.15,14.6,131.2865,...,89.450002,131.246999,260.000000,0.08,0.0,0.0,0,0,None,0.0
5f6eeb3e5d20e300018cb3da,2020-09-01 23:57:00,2020-09-01 23:58:00,60000,60000,Day,BM 1,86.75,11.15,14.6,131.2865,...,89.450002,131.246999,243.333333,0.08,0.0,0.0,0,0,None,720.0
5f6eeb3e5d20e300018cb3d9,2020-09-01 23:56:00,2020-09-01 23:57:00,60000,60000,Day,BM 1,86.75,11.15,14.6,131.2865,...,89.450002,131.246999,240.000000,0.08,0.0,0.0,0,0,None,2180.0
5f6eeb3e5d20e300018cb3d8,2020-09-01 23:55:00,2020-09-01 23:56:00,60000,60000,Day,BM 1,86.75,11.15,14.6,131.2865,...,89.450002,131.246999,240.000000,0.08,0.0,0.0,0,0,None,740.0


# Selecting columns and silencing the `_only` Warning

To select a specific set of columns, provide a list of column names as a value for the key _only.  For example, `'_only': ['column1', 'column2', 'column3']`

If you do not use _only, the SDK will automatically select the first 50 stats in the machine's configuration, plus common metadata fields for the query.  

Note, you can also pass `'_only': '*'`, which will return everything, including a large number of internal fields.  Since this includes may fields you probably will not need, expect the resulting queries to be quite slow.

**IMPORTANT** If a selected column is all null, it will not be included in the returned data frame.  If you are getting fewer columns returned than expected, this mostly likely means that there was only null data for that column.



In [4]:
# Get the first 10 columns, plus Machine and End Time
select_columns = ['Machine', 'End Time'] + columns[:5]

query = {'Machine': machines[0],
         'End Time__gte' : datetime(2020, 9, 1), 
         'End Time__lte' : datetime(2020, 9, 2), 
         '_order_by': '-End Time',
         '_only': select_columns}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')
df.head()

Size of returned data: (1441, 7)


,End Time,Machine,BM: Axial Load,BM: Dome Depth,BM: Thickwall Variation,BM: Can Weight,BM: Thickwall Avg
id,,,,,,,
5f6eeb3e5d20e300018cb3dc,2020-09-02 00:00:00,BM 1,86.75,11.15,14.6,10.28,149.833328
5f6eeb3e5d20e300018cb3db,2020-09-01 23:59:00,BM 1,86.75,11.15,14.6,10.28,149.833328
5f6eeb3e5d20e300018cb3da,2020-09-01 23:58:00,BM 1,86.75,11.15,14.6,10.28,149.833328
5f6eeb3e5d20e300018cb3d9,2020-09-01 23:57:00,BM 1,86.75,11.15,14.6,10.28,149.833328
5f6eeb3e5d20e300018cb3d8,2020-09-01 23:56:00,BM 1,86.75,11.15,14.6,10.28,149.833328


## Restricting the number of rows returned with `_limit` and `_offset`

To restrict the number of rows, use the _limit query option.  For example, `'_limit': 500`.  This will then return at most 500 rows.  

To skip over a specified number of rows, use the _offset query option.  For example `'_offset': 50`.

It is fairly common to use a combination of _limit and _offset togheter for applications such as paginating data.  For example, if a query would normally return 100 rows and you want to break it into two queries you could return the first 50 rows with `'_offset': 0, '_limit': 50` and then return the second 50 rows with `'_offset': 50, '_limit': 50`.

In [5]:
query = {'Machine': machines[0],
         'End Time__gte' : datetime(2020, 9, 1), 
         'End Time__lte' : datetime(2020, 9, 2), 
         '_order_by': '-End Time',
         '_offset': 10,
         '_limit': 500}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')

# Notice in the returned data set that the first row is at 23:50 instead of midnight, becuase of the offset
df.head()

_only not specified.  Selecting first 50 fields.
Size of returned data: (500, 40)


,Start Time,End Time,Cycle Time (Net),Cycle Time (Gross),Shift,Machine,BM: Axial Load,BM: Dome Depth,BM: Thickwall Variation,BM: Trim Height Max,...,BM: Thinwall Avg,BM: Trim Height Avg,0_BM: CPM,BM: Trim Height Variation,0_BM: Primary Tear Off PPM,BM: Trimmer Jam - Data,BM: Trimmer Jam Time,BM: Trimmer Jam Events,BM: Cans Out,BM: Cans Out Total
id,,,,,,,,,,,,,,,,,,,,,
5f6eeb3d5d20e300018cb3d2,2020-09-01 23:49:00,2020-09-01 23:50:00,60000,60000,Day,BM 1,86.75,11.15,14.6,131.2865,...,89.450002,131.246999,240.0,0.08,0.0,0.0,0,0,None,4280.0
5f6eeb3d5d20e300018cb3d1,2020-09-01 23:48:00,2020-09-01 23:49:00,60000,60000,Day,BM 1,86.75,11.15,14.6,131.2865,...,89.450002,131.246999,240.0,0.08,0.0,0.0,0,0,None,0.0
5f6eeb3d5d20e300018cb3d0,2020-09-01 23:47:00,2020-09-01 23:48:00,60000,60000,Day,BM 1,86.75,11.15,14.6,131.2865,...,89.450002,131.246999,252.0,0.08,0.0,0.0,0,0,None,3440.0
5f6eeb3d5d20e300018cb3cf,2020-09-01 23:46:00,2020-09-01 23:47:00,60000,60000,Day,BM 1,86.75,11.15,14.6,131.2865,...,89.450002,131.246999,260.0,0.08,0.0,0.0,0,0,None,0.0
5f6eeb3d5d20e300018cb3ce,2020-09-01 23:45:00,2020-09-01 23:46:00,60000,60000,Day,BM 1,86.75,11.15,14.6,131.2865,...,89.450002,131.246999,248.0,0.08,0.0,0.0,0,0,None,840.0


# Data from more than one Machine or filtering by a list of values using `__in`

Filters can specify a list of acceptable values.  This is most commonly used when selecting data from more than one machine, though it can be used on any field name.  This is done by appending `__in` (*note two underscores*) to the column name and then specifying the list of options.  For example:

    'Machine__in': ['Oven1', 'Oven2']

or

    'Status__in': ['Idle', 'Maintenance', 'Down']

**Important** Selecting multiple machines of different types can result in spare and confusing data frames.  It is strongly recommended to only pick multiple machines of the same type.

You can also query for values that are not in a list by using `__nin` with the same format as `__in`.  For example:

    'Product_Code__nin': ['SuperMax 5000', 'MegaValue 6000']

In [6]:
# Note: taking the first three machines' data, so will result in three times as many records returned
query = {'Machine__in': machines[0:3],
         'End Time__gte' : datetime(2020, 9, 1), 
         'End Time__lte' : datetime(2020, 9, 2), 
         '_order_by': '-End Time'}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')
# Notice the Machine column now has three different values
df.head()

_only not specified.  Selecting first 50 fields.
Size of returned data: (4323, 40)


,Start Time,End Time,Cycle Time (Net),Cycle Time (Gross),Shift,Machine,BM: Axial Load,BM: Dome Depth,BM: Thickwall Variation,BM: Trim Height Max,...,BM: Thinwall Avg,BM: Trim Height Avg,0_BM: CPM,BM: Trim Height Variation,0_BM: Primary Tear Off PPM,BM: Trimmer Jam - Data,BM: Trimmer Jam Time,BM: Trimmer Jam Events,BM: Cans Out,BM: Cans Out Total
id,,,,,,,,,,,,,,,,,,,,,
5f6ef02c802e110001f0775c,2020-09-01 23:59:00,2020-09-02 00:00:00,60000,60000,Day,BM 3,103.75,11.19,3.500000,131.24666,...,89.866666,131.214828,244.000000,0.06,0.0,0.0,0,0,None,2920.0
5f6eebd7d27a3400012b351e,2020-09-01 23:59:00,2020-09-02 00:00:00,60000,60000,Day,BM 2,99.81,11.28,13.033333,131.28000,...,89.899999,131.236165,240.000000,0.07,0.0,0.0,0,0,None,2920.0
5f6eeb3e5d20e300018cb3dc,2020-09-01 23:59:00,2020-09-02 00:00:00,60000,60000,Day,BM 1,86.75,11.15,14.600000,131.28650,...,89.450002,131.246999,256.000000,0.08,0.0,0.0,0,0,None,2920.0
5f6ef02c802e110001f0775b,2020-09-01 23:58:00,2020-09-01 23:59:00,60000,60000,Day,BM 3,103.75,11.19,3.500000,131.24666,...,89.866666,131.214828,260.000000,0.06,0.0,0.0,0,0,None,0.0
5f6eebd7d27a3400012b351d,2020-09-01 23:58:00,2020-09-01 23:59:00,60000,60000,Day,BM 2,99.81,11.28,13.033333,131.28000,...,89.899999,131.236165,243.333333,0.07,0.0,0.0,0,0,None,0.0


# Filtering to only rows where a specified field exists with `__exists`

Some data fields, such as inspection data, are often quite sparse.  To filter to only rows with or without non-null values, use `__exists`.  `__exists` should be appended to the name of the field, and then give it a boolean for if you want the field to exist (True) or not exist (False).  For example:

    'Inspection_Value__exists': True

or

    'Failure_Code__exists': False

In [7]:

query = {'Machine__in': ['BM 1', 'F2 BM 4'],
         'BM: Axial Load__exists': True,
         '_order_by': '-End Time',
         '_only': ['Machine', 'End Time', 'BM: Axial Load'],
         '_limit': 100}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')
# Query was for two machines, but BM: Axial Load not on the second, so that doesn't appear in the data set
df.head()

Size of returned data: (100, 3)


,End Time,Machine,BM: Axial Load
id,,,
6023eee3db18e80001cacc3a,2021-02-10 14:34:00,BM 1,93.00
6023eeaadb18e80001cacba6,2021-02-10 14:33:00,BM 1,93.00
6023ee6bdb18e80001cacb03,2021-02-10 14:32:00,BM 1,93.00
6023ee35db18e80001caca7c,2021-02-10 14:31:00,BM 1,93.00
6023edf3db18e80001cac9c9,2021-02-10 14:30:00,BM 1,105.73


# Testing for inequality with `__ne`

The standard `key: value` format assumes it is testing when the key equals the value.  To change this to inequality, add a `__ne` suffix.  For example, `'StatusCode__ne': 0`



In [8]:
query = {'Machine': machines[0],
         'BM: Axial Load__ne': 86.75,
         'End Time__gte' : datetime(2020, 9, 1), 
         'End Time__lte' : datetime(2020, 9, 2), 
         '_order_by': '-End Time'}
df = cli.get_cycles(**query)

print(f'Size of returned data: {df.shape}')
df.head()

_only not specified.  Selecting first 50 fields.
Size of returned data: (1129, 40)


,Start Time,End Time,Cycle Time (Net),Cycle Time (Gross),Shift,Machine,BM: Axial Load,BM: Dome Depth,BM: Thickwall Variation,BM: Trim Height Max,...,BM: Thinwall Avg,BM: Trim Height Avg,0_BM: CPM,BM: Trim Height Variation,0_BM: Primary Tear Off PPM,BM: Trimmer Jam - Data,BM: Trimmer Jam Time,BM: Trimmer Jam Events,BM: Cans Out,BM: Cans Out Total
id,,,,,,,,,,,,,,,,,,,,,
5f6eeb195d20e300018cb29b,2020-09-01 18:47:00,2020-09-01 18:48:00,60000,60000,Night,BM 1,88.26,11.16,5.5,131.294,...,89.075001,131.255753,240.000000,0.08,0.0,0.0,0,0,None,220.0
5f6eeb195d20e300018cb29a,2020-09-01 18:46:00,2020-09-01 18:47:00,60000,60000,Night,BM 1,88.26,11.16,5.5,131.294,...,89.075001,131.255753,248.000000,0.08,0.0,0.0,0,0,None,1480.0
5f6eeb195d20e300018cb299,2020-09-01 18:45:00,2020-09-01 18:46:00,25603,60000,Night,BM 1,88.26,11.16,5.5,131.294,...,89.075001,131.255753,86.666667,0.08,0.0,0.0,0,0,None,2640.0
5f6eeb195d20e300018cb298,2020-09-01 18:44:00,2020-09-01 18:45:00,0,60000,Night,BM 1,88.26,11.16,5.5,131.294,...,89.075001,131.255753,0.000000,0.08,0.0,0.0,0,0,None,440.0
5f6eeb185d20e300018cb297,2020-09-01 18:43:00,2020-09-01 18:44:00,0,60000,Night,BM 1,98.71,11.16,5.5,131.294,...,89.075001,131.255753,0.000000,0.08,0.0,0.0,0,0,None,320.0
